In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm, chi2
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.interpolate import interp1d

# -----------------------------
# ODE model
# -----------------------------
def ode_rhs(t, y, x):
    dydt1 = 0.39562*y[0]*(1-y[0]/890447238) - 1.06870e-07*y[0]*y[1] - x[0]*y[0]*y[2]
    dydt2 = 75198 - 0.00122*y[1] -1.59893e-09*y[0]*y[1]
    dydt3 = x[1] - x[2]*y[2] - x[3]*y[0]*y[2]
    return [dydt1, dydt2, dydt3]

def solve_dataset(t_obs, x, y10, y20, y30):
    sol = solve_ivp(lambda t, y: ode_rhs(t, y, x),
                    (t_obs[0], t_obs[-1]),
                    [y10, y20, y30],
                    t_eval=t_obs, method="BDF", rtol=1e-6, atol=1e-8)
    return sol.y[0]  # compare y1 to experimental data

# -----------------------------
# Experimental data
# -----------------------------
datasets = [
    (np.array([18,20,22,24,26,28,30,32], float),
     np.array([1.51e6,1.642e7,3.019e7,6.159e7,1.1406e8,1.8944e8,3.1565e8,4.7166e8], float),
    1432029.87),
    (np.array([10,12,14,16,18,20,22,24,26], float),
     np.array([2.02e6,6.61e6,4.046e7,1.1102e8,2.4265e8,4.7158e8,7.7838e8,1.17088e9,1.26995e9], float),
      14527932.65),
    (np.array([6,8,10,12,14,16,18,20], float),
     np.array([6.23e6,4.28e7,1.0936e8,2.1472e8,4.2609e8,7.2969e8,1.11062e9,1.376525e9], float),
    13558129.72),
    (np.array([2,4,6,8,10,12,14], float),
     np.array([2.244e7,8.769e7,2.3401e8,4.4702e8,7.2802e8,1.13022e9,1.50725e9], float),
     36022479.73)
]
y20 = 94088.81
y30 = 51639.71

theta_true = np.array([1.0000003423976e-07, 1.09944329169823e-07,  0.63506536118549,9.99991261886903e-07])

# -----------------------------
# Generate noisy data
# -----------------------------
CV, SigmaFloor = 0.05, 1e5
exp_data, sigma_data = [], []
for t_obs, vals, y10 in datasets:
    clean = solve_dataset(t_obs, theta_true, y10, y20, y30)
    sigma_vals = np.maximum(CV*clean, SigmaFloor)
    noisy = clean + np.random.normal(0, sigma_vals)
    exp_data.append(noisy)
    sigma_data.append(sigma_vals)

print("Noisy experimental data generated.")

# -----------------------------
# Log-likelihood
# -----------------------------
def loglikelihood(theta):
    ll = 0.0
    for (t_obs, data, y10), noisy, sigmas in zip(datasets, exp_data, sigma_data):
        pred = solve_dataset(t_obs, theta, y10, y20, y30)
        resid = noisy - pred
        ll += np.sum(norm.logpdf(resid, 0, sigmas))
    return ll

def neg_ll(theta):
    return -loglikelihood(theta)

# -----------------------------
# Bounds
# -----------------------------
bounds = [(1e-9,1e-6), (1e-8,1e-4), (1e-3,1), (1e-8,1e-4)]

# -----------------------------
# Estimate MLE
# -----------------------------
print("Running optimization...")
res_global = differential_evolution(neg_ll, bounds, maxiter=40, polish=False, workers=-1)
res_local = minimize(neg_ll, res_global.x, bounds=bounds, method="L-BFGS-B")
theta_mle = res_local.x
print("MLE:", theta_mle)

# -----------------------------
# Profile likelihood
# -----------------------------
def profile_likelihood(param_index, theta_mle, bounds, n_grid=25):
    p_grid = np.linspace(bounds[param_index][0], bounds[param_index][1], n_grid)
    ll_values = np.empty_like(p_grid)
    ll_mle = loglikelihood(theta_mle)
    nuisance_guess = [theta_mle[i] for i in range(len(theta_mle)) if i != param_index]
    for j, val in enumerate(tqdm(p_grid, desc=f"Param {param_index+1}")):
        theta_fixed = theta_mle.copy()
        theta_fixed[param_index] = val
        free_idx = [i for i in range(len(theta_mle)) if i != param_index]
        free_bounds = [bounds[i] for i in free_idx]
        def neg_ll_nuisance(free_params):
            theta_var = theta_fixed.copy()
            theta_var[free_idx] = free_params
            return -loglikelihood(theta_var)
        res = minimize(neg_ll_nuisance, nuisance_guess, bounds=free_bounds,
                       method="L-BFGS-B", options={"maxiter":200})
        ll_values[j] = -res.fun
        nuisance_guess = res.x
    norm_ll = ll_values - ll_mle
    return param_index, p_grid, norm_ll

# -----------------------------
# Run all profiles
# -----------------------------
results = Parallel(n_jobs=4)(
    delayed(profile_likelihood)(i, theta_mle, bounds, n_grid=25)
    for i in range(4)
)

# -----------------------------
# Plot
# -----------------------------
cutoff = -chi2.ppf(0.95, df=1)/2
labels = [r"$\phi_m$", r"$\gamma_m$", r"$\delta_m$", r"$\eta_m$"]

fig, axes = plt.subplots(1, 3, figsize=(7,8))
axes = axes.ravel()
cutoff = -chi2.ppf(0.95, df=1)/2

for param_index, p_grid, norm_ll in results:
    ax = axes[param_index]
    ax.plot(p_grid, norm_ll, color='blue', lw=2, label="Profile")
    ax.plot(p_grid, norm_ll, 'ro', markeredgecolor='black', label="Steps")
    ax.axhline(cutoff, color='green', linestyle='--', label='95% CI cutoff')
    ax.axvline(theta_mle[param_index], color='black', linestyle=':', label='MLE')
    ax.set_xlabel(param_labels[param_index])
    ax.set_ylabel(r"$\Delta$ log-likelihood")
    ax.legend()
plt.tight_layout()
plt.show()
